# Download specific genomes from NCBI FTP Site

In [41]:
## IMPORT ##
import pandas as pd
from pandas import read_csv
import urllib
import urllib.request
import time
import gzip
import shutil
import shutil

In [ ]:
seq = pd.read_csv('sequences.csv') #sequences from NCBI Virus -> assemblies

In [40]:
#new dataframe with individual columns
new_seq = seq[['Assembly', 'Species', 'Molecule_type', 'Family', 'Host', 'GenBank_Title']].copy()

#sorting values by Host and Family
new_seq.sort_values(by=['Host', 'Family'])

#searching Family Siphoviridae which Host is Lactococcus lactis
siphoviridae = new_seq.loc [(new_seq['Family'] == 'Siphoviridae') & (new_seq['Host'] == 'Lactococcus lactis')] 

#Family Siphoviridae|Lactococcus Lactis Host assemblies -> making list 
asb = siphoviridae["Assembly"].tolist()

In [ ]:
#download assembly_summary_refseq txt file to get the path from specific assemblies
urllib.request.urlretrieve("https://ftp.ncbi.nlm.nih.gov/genomes/refseq/assembly_summary_refseq.txt", "assembly_summary_refseq.txt")

In [ ]:
def get_assemblies(phages_list):
    assembly_sum = pd.read_csv('assembly_summary_refseq.txt', sep='\t', skiprows=1) #import assembly_summary_refseq file to dataframe
    #names of columns
    assembly_sum.columns = ["assembly_accession", "bioproject", "biosample", "wgs_master", "refseq_category", "taxid", "species_taxid", "organism_name", "infraspecific_name", "isolate", "version_status", "assembly_level", "release_type", "genome_rep", "seq_rel_date", "asm_name", "submitter", "gbrs_paired_asm", "paired_asm_comp", "ftp_path", "excluded_from_refseq", "relation_to_type_material"]

    for assembly in phages_list:
        # searching specific genomes
        my_df = assembly_sum[(assembly_sum['assembly_accession'] == assembly)] & [(assembly_sum['version_status'] == 'latest')] 
        my_df = my_df[['ftp_path','assembly_accession','asm_name']]

        #Process the newly created file and download genomes from NCBI website
        ftp = my_df['ftp_path'].tolist() #making ftp list
        asm = my_df['asm_name'].to_list() #making asm list
        for elem in ftp:
            try: 
                file = str(assembly)
                for i in asm:
                    file_suffix='/'+assembly+'_'+i+'_genomic.fna.gz'
                #file_suffix = str(file_suffix)
                file_in = assembly + '.fna.gz'
                file_out = assembly + '.fna' #fasta format
                download = urllib.request.urlretrieve(elem+file_suffix, file_in)
                print("Download:", file_in)
                time.sleep(1)
            except:        
                print("-I- Skipping",file_in," - doesn't exist.")
            #unzip files 
            with gzip.open(file_in, 'rb') as f_in:
                with open(file_out, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

    return
